# Cargo Delivery Route Optimization (Timothy Manolias)

### The following program calculates the optimal delivery route a freight company should pursue in order to minimize the time traveled to 68 cities in the United States.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML

Image(url='Images/Question.png', width=700)

In [2]:
'''Imports Libraries and Data.'''

import pandas as pd
import numpy as np
import folium
from gurobipy import *

cargo_city_locations = pd.read_csv('Data/cargo-city-locations.csv')
cargo_city_locations.head()

,State,City,Latitude,Longitude
0,Alabama,Montgomery,32.377716,-86.300568
1,Alaska,Juneau,58.301598,-134.420212
2,Arizona,Phoenix,33.448143,-112.096962
3,Arkansas,Little Rock,34.746613,-92.288986
4,California,Sacramento,38.576668,-121.493629


### Data Preprocessing & Feature Engineering

In [3]:
'''Calculates Each Pair of Cities.'''

cargo_city_locations['Latitude_Radians'] = cargo_city_locations['Latitude'] / 360 * 2 * np.pi
cargo_city_locations['Longitude_Radians'] = cargo_city_locations['Longitude'] / 360 * 2 * np.pi

cities, city_combos = cargo_city_locations['City'].values, []
for first_city in cities:
    for second_city in cities:
        if first_city != second_city:
            city_combos.append((first_city, second_city))

In [4]:
'''Calculates Distance and Travel Time Between Each City-Pair.'''

r = 6378.137 #Radius of Earth (km)
aircraft_speed = 908 #(km/hour)

city_combos_df = pd.DataFrame()
for i, combo in enumerate(city_combos):
    # City 1
    city_1 = combo[0]
    city_combos_df.at[i, 'City1'] = city_1
    city_combos_df.at[i, 'City1_Lat'] = cargo_city_locations.loc[cargo_city_locations['City'] == city_1]['Latitude_Radians'].values[0]
    city_combos_df.at[i, 'City1_Long'] = cargo_city_locations.loc[cargo_city_locations['City'] == city_1]['Longitude_Radians'].values[0]
    
    # City 2
    city_2 = combo[1]
    city_combos_df.at[i, 'City2'] = city_2
    city_combos_df.at[i, 'City2_Lat'] = cargo_city_locations.loc[cargo_city_locations['City'] == city_2]['Latitude_Radians'].values[0]
    city_combos_df.at[i, 'City2_Long'] = cargo_city_locations.loc[cargo_city_locations['City'] == city_2]['Longitude_Radians'].values[0]
    
    
    # Haversine Formula: Calculates angle corresponding to the great circle distance between cities 1 and 2
    lat_i = city_combos_df.at[i, 'City1_Lat']
    lat_j = city_combos_df.at[i, 'City2_Lat']
    lon_i = city_combos_df.at[i, 'City1_Long']
    lon_j = city_combos_df.at[i, 'City2_Long']
    city_combos_df.at[i, 'H'] = ((1 - np.cos(lat_j - lat_i)) / 2)  +  (np.cos(lat_i) * np.cos(lat_j))  *  ((1 - np.cos(lon_j - lon_i)) / 2)
    
    # Calculates distance between city pair
    h = city_combos_df.at[i, 'H']
    city_combos_df.at[i, 'Distance (km)'] = 2 * r * np.arcsin(np.sqrt(h))
    
    # Calculates travel time
    distance = city_combos_df.at[i, 'Distance (km)']
    city_combos_df.at[i, 'Travel_Time (hrs)'] = distance / aircraft_speed

### Calculating Schedules

**Average Total Travel Time (In Hours) for 100 Randomly Generated Sequences:**

In [5]:
def total_travel_time(city_order):
    '''Calulates the total travel time, given a sequence of cities.'''
    
    # Calculates travel time for each consecutive city pair
    travel_times = []
    for i in range(len(city_order) - 1):
        # Gets consecutive-city names
        city_A = cargo_city_locations.iloc[city_order[i]]['City']
        city_B = cargo_city_locations.iloc[city_order[i+1]]['City']
        
        # Calulates travel time between cities
        travel_time = city_combos_df.loc[ (city_combos_df['City1'] == city_A) &
                                          (city_combos_df['City2'] == city_B) ]['Travel_Time (hrs)'].values[0]
        
        travel_times.append(travel_time)
        
    # Appends travel time for last city back to origin
    last_city = cargo_city_locations.iloc[city_order[-1]]['City']
    first_city = cargo_city_locations.iloc[city_order[0]]['City']
    final_travel_time = city_combos_df.loc[ (city_combos_df['City1'] == last_city) &
                                            (city_combos_df['City2'] == first_city) ]['Travel_Time (hrs)'].values[0]
    
    travel_times.append(final_travel_time)
    
    
    return sum(travel_times)

In [6]:
'''Calculates Average of 100 Random Sequences of 68 Cities.'''

np.random.seed(50)

nCities = 68
seq_travel_times = []

for seq in range(100):
    city_order = np.random.permutation(nCities)
    seq_travel_time = total_travel_time(city_order)
    seq_travel_times.append(seq_travel_time)
    
avg_travel_times = np.mean(seq_travel_times)

print(f'Average Travel Time: {avg_travel_times:.2f} hours')

Average Travel Time: 149.46 hours


**Total Travel Time of the Closest-City Sequence:**
- Starting from Los Angeles, the next city in the schedule is the one that is closest to the current city in travel time and has not been visited yet.

In [7]:
'''Maps Cities to Corresponding Index.
   Assigns LA as First City.'''

# Creates index list of all possible cities
city_mappings = {}
for i, row in cargo_city_locations.iterrows():
    city_mappings[row['City']] = i
possible_inds = np.arange(68)

# Adds LA as first city and removes LA from possible_inds
LA_ind = cargo_city_locations.index[cargo_city_locations['City'] == 'Los Angeles'].values[0]
city_order = [LA_ind]
possible_inds = np.delete(possible_inds, np.argwhere(possible_inds == city_order[-1]))

In [8]:
'''Appends Next-Closest City to Route Until There Are No Cities Left.'''

while len(possible_inds) > 0:
    current_city = cargo_city_locations.iloc[city_order[-1]]['City']
    
    # Limits combos_df to entries of current city
    curr_city_df = city_combos_df.loc[city_combos_df['City1'] == current_city]

    # Sorts curr_city_df by shortest travel time
    curr_travel_time_sort = curr_city_df.sort_values(by=['Travel_Time (hrs)'])
    
    # Obtains indices of shortest cities list
    closest_cities = curr_travel_time_sort['City2'].values
    closest_cities_inds = [city_mappings[city] for city in closest_cities]

    # Finds next closest city that was not already visited
    for city in closest_cities_inds:
        if city in possible_inds:
            # Adds city to route
            city_order.append(city)

            # Removes city from possible_inds
            possible_inds = np.delete(possible_inds, np.argwhere(possible_inds == city))

            break

In [9]:
'''Prints City-Order list.'''

index_to_city = {val: key for key, val in city_mappings.items()}
city_name_order = [index_to_city[city] for city in city_order] + ['Los Angeles']
', '.join([city for city in city_name_order])

'Los Angeles, San Diego, Phoenix, El Paso, Santa Fe, Denver, Cheyenne, Pierre, Bismarck, St. Paul, Madison, Chicago, Indianapolis, Frankfort, Columbus, Charleston, Charlotte, Columbia, Raleigh, Richmond, Washington, Annapolis, Dover, Philadelphia, Trenton, New York City, Hartford, Providence, Boston, Concord, Montpelier, Albany, Augusta, Harrisburg, Detroit, Lansing, Springfield, Jefferson City, Topeka, Lincoln, Des Moines, Oklahoma City, Fort Worth, Dallas, Austin, San Antonio, Houston, Baton Rouge, Jackson, Little Rock, Nashville, Atlanta, Montgomery, Tallahassee, Jacksonville, Salt Lake City, Boise, Helena, Seattle, Olympia, Portland, Salem, Carson City, Sacramento, San Francisco, San Jose, Juneau, Honolulu, Los Angeles'

In [10]:
print(f'Total Travel Time: {total_travel_time(city_order):.2f} hours')

Total Travel Time: 35.85 hours


In [11]:
'''Displays Map of Closest-City Route.'''

avg_latitude = cargo_city_locations['Latitude'].mean()
avg_longitude = cargo_city_locations['Longitude'].mean()
route_map = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=4, detect_retina=True)

# Creates markers for each location
for i in range(cargo_city_locations.shape[0]):
    marker_color = 'red' if i == 51 else 'blue'
    folium.CircleMarker([cargo_city_locations['Latitude'][i], cargo_city_locations['Longitude'][i]], 
                        popup=cargo_city_locations['City'][i],
                        radius = 6,
                        color = marker_color,
                        fill = True,
                        fillcolor = 'blue').add_to(route_map)

# Adds Los Angeles as last stop
city_order.append(51)

# Adds lines to indicate route
route = list(zip(cargo_city_locations['Latitude'][city_order], cargo_city_locations['Longitude'][city_order])) 
poly = folium.PolyLine(route, color="green", weight=2.5, opacity=1)
poly.add_to(route_map)

route_map

### Solving the Optimal Route

**Calculates the Order in Which the Cities Should Be Visited, so as to Minimize the Total Travel Time.**

In [12]:
def getSubtours(route_sequence):
    '''Obtains All Subtours of a Given Route Sequence.'''
    
    subtour_list = []
    unvisited = list(range(nCities))
    
    while len(unvisited) > 0:
        node = unvisited.pop()
        
        subtour = []
        subtour.append(node)
        
        next_node = list(filter(lambda t: t[0] == node, route_sequence))[0][1]
        
        while next_node in unvisited:
            subtour.append(next_node)
            unvisited.remove(next_node)
            next_node = list(filter(lambda t: t[0] == next_node, route_sequence))[0][1]
            
        subtour_list.append(subtour)
    
    return subtour_list

In [13]:
def eliminateSubtours(model, where):
    '''Checks if Any Subtours Exist and Applies Necessary Constraints to Model if So.'''
    
    if where == GRB.Callback.MIPSOL:
        x_val = model.cbGetSolution(x)
        route_sequence = [ (i,j) for (i,j) in city_ind_combos if x_val[i,j] > 0.5]
        subtour_list = getSubtours(route_sequence)
        if (len(subtour_list) > 1):
            for subtour in subtour_list:
                model.cbLazy( sum(x[i,j] for i in subtour for j in subtour if i != j) <= len(subtour) - 1)

In [14]:
'''Maps Each City-Index Pair to its Corresponding Travel Time.'''

# Obtains index combos for each city pair.
city_inds, city_ind_combos = cargo_city_locations.index.values.tolist(), []
for first_city in city_inds:
    for second_city in city_inds:
        if first_city != second_city:
            city_ind_combos.append((first_city, second_city))

# Maps city-index pairs
travel_time_dict = {}
for comb in city_ind_combos:
    city_1 = cargo_city_locations.iloc[comb[0]]['City']
    city_2 = cargo_city_locations.iloc[comb[1]]['City']
    
    travel_time = city_combos_df.loc[ (city_combos_df['City1'] == city_1) &
                                      (city_combos_df['City2'] == city_2) ]['Travel_Time (hrs)'].values[0]
    
    travel_time_dict[comb] = travel_time

In [15]:
'''Creates Model and Calculates Optimal Route.'''

nCities = cargo_city_locations.shape[0]

# Creates model
m = Model()

# Suppresses output
m.Params.outputFlag = 0

# Adds decision variables
x = m.addVars(city_ind_combos, vtype=GRB.BINARY)

# Adds constraints
for i in range(nCities):
    m.addConstr( sum(x[i,j] for j in range(nCities) if j != i ) == 1)
    m.addConstr( sum(x[j,i] for j in range(nCities) if j != i ) == 1)

# Adds objective function
m.setObjective(sum( travel_time_dict[i,j] * x[i,j] for (i,j) in city_ind_combos ), GRB.MINIMIZE)

m.update()

# Enables lazy constraints
m.params.LazyConstraints = 1

# Supplies the callback to Gurobi:
m.optimize(eliminateSubtours)

Academic license - for non-commercial use only - expires 2022-01-16
Using license file /Users/Nolias/gurobi.lic


In [16]:
'''Outputs Optimal Route Solution.'''

# Prints travel time
print(f'Total Travel Time (in hours) of Optimal Route: {m.obj_val:.2f} hours')

# Prints route
route_sequence = [ (i,j) for (i,j) in city_ind_combos if x[i,j].x > 0.5]
subtour_list = getSubtours(route_sequence)
optimal_route = subtour_list[0]
optimal_route.append( optimal_route[0] )
print('\nOptimal Route:')
print(', '.join([index_to_city[c] for c in optimal_route]))

Total Travel Time (in hours) of Optimal Route: 30.55 hours

Optimal Route:
Portland, Olympia, Seattle, Juneau, Honolulu, San Francisco, San Jose, Sacramento, Carson City, Los Angeles, San Diego, Phoenix, El Paso, Santa Fe, Oklahoma City, Dallas, Fort Worth, Austin, San Antonio, Houston, Baton Rouge, Jackson, Little Rock, Jefferson City, Topeka, Lincoln, Des Moines, Springfield, Indianapolis, Frankfort, Nashville, Atlanta, Montgomery, Tallahassee, Jacksonville, Columbia, Charlotte, Raleigh, Richmond, Washington, Annapolis, Dover, Philadelphia, Trenton, New York City, Hartford, Providence, Boston, Concord, Augusta, Montpelier, Albany, Harrisburg, Charleston, Columbus, Detroit, Lansing, Chicago, Madison, St. Paul, Bismarck, Pierre, Cheyenne, Denver, Salt Lake City, Helena, Boise, Salem, Portland


In [17]:
'''Displays Map of Optimal Route.'''

optimal_route_map = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=4, detect_retina=True)

# Creates markers for each location
for i in range(cargo_city_locations.shape[0]):
    marker_color = 'red' if i == optimal_route[0] else 'blue'
    folium.CircleMarker([cargo_city_locations['Latitude'][i], cargo_city_locations['Longitude'][i]], 
                        popup=cargo_city_locations['City'][i],
                        radius = 6,
                        color = marker_color,
                        fill = True,
                        fillcolor = 'blue').add_to(optimal_route_map)

# Adds lines to indicate route
optimal_route = list(zip(cargo_city_locations['Latitude'][city_order], cargo_city_locations['Longitude'][city_order])) 
poly = folium.PolyLine(route, color="green", weight=2.5, opacity=1)
poly.add_to(optimal_route_map)

optimal_route_map